In [14]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
import json

In [21]:
experiments = ["original", "random_20_5k", "random_10", "changing_template", "instruction_last", "instruction_sandwich"]


In [22]:
_temp = []
for e in experiments:
    print(e)

    for file in os.listdir(f"../{e}/datamodels/collections/train/"):
        if file.startswith("groundtruth"):
            _temp.append(pl.read_ipc(f"../{e}/datamodels/collections/train/{file}", memory_map=False).drop("input").with_columns(pl.lit(e).alias("experiment")))
collections = pl.concat(_temp)

original
random_20_5k
random_10
changing_template
instruction_last
instruction_sandwich


In [23]:
collections.filter(pl.col("evaluation")>0).group_by(pl.col("experiment")).agg((pl.count()).alias("recall"))

/tmp/ipykernel_496861/1982936945.py:1: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  collections.filter(pl.col("evaluation")>0).group_by(pl.col("experiment")).agg((pl.count()).alias("recall"))


experiment,recall
str,u32
"""original""",126102
"""instruction_last""",50707
"""changing_template""",39174
"""random_20_5k""",388310
"""random_10""",154429


In [24]:
collections.filter(pl.col("evaluation")>0).select(["experiment", "test_idx"]).unique().group_by(pl.col("experiment")).agg(pl.count())

/tmp/ipykernel_496861/403960541.py:1: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  collections.filter(pl.col("evaluation")>0).select(["experiment", "test_idx"]).unique().group_by(pl.col("experiment")).agg(pl.count())


experiment,count
str,u32
"""instruction_last""",320
"""changing_template""",358
"""random_10""",353
"""original""",358
"""random_20_5k""",369


In [25]:
collections.filter(pl.col("evaluation")>0).group_by(["experiment", "test_idx"]).agg(pl.count()).sort("experiment", "test_idx").filter(pl.col("count")>10)

/tmp/ipykernel_496861/58300571.py:1: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  collections.filter(pl.col("evaluation")>0).group_by(["experiment", "test_idx"]).agg(pl.count()).sort("experiment", "test_idx").filter(pl.col("count")>10)


experiment,test_idx,count
str,i64,u32
"""changing_template""",0,37
"""changing_template""",2,62
"""changing_template""",4,15
"""changing_template""",6,83
"""changing_template""",9,39
…,…,…
"""random_20_5k""",493,278
"""random_20_5k""",494,14
"""random_20_5k""",497,29
